In [1]:
from pImpactR import MLI as mli
import pandas as pd
import numpy as np

In [4]:
rawlines = mli._file2rawlines(fname='mli.in')
rawlines[1:-6]

['beam:beam,particle=electron,energy=511000.1',
 'units:units,type=static',
 'setdefaults:globaldefaults,lfrngsbend=0,tfrngsbend=0,lfrngquad=0,tfrngquad=0,driftexact=0',
 'stm:stm,nmap=1',
 'gtm:gtm,nmap=1,iopt=2',
 'tmo:tmo,ifile=17',
 'raytrace:raytrace,type=readonly,file1=rays.in',
 'particledump:particledump,file=rays.out,precision=15,close=false,flush=true,nunits=0',
 'autotrack:autotrack,type=symplectic5',
 'clear:iden',
 'fin:end',
 'nlinsert:nlinsert,zstart=0.0,zend=1.8,steps=1000,zlen=1.8,k=1.45446332708327,c=0.01,tau=-0.4',
 'tmi:tmi,iopt=2,ifile=16,nopt=1,nskip=0',
 'linedump, line=(nlinsert gtm particledump ) ']

In [5]:
def _isint(s):
  try:
    int(s)
    return True
  except ValueError:
    return False

  
def _isfloat(s):
  try:
    float(s.replace('D','E',1).replace('d','e',1))
    return True
  except ValueError:
    return False

  
def _isvariable(s,variables):
  if s in variables.keys():
    return True
  else:
    return False

  
def _str2val(s,variables):
  if _isint(s):
    return int(s)
  elif _isfloat(s):
    return float(s)
  elif _isvariable(s,variables):
    return variables[s]
  else:
    return s

In [6]:
def _rawlines2var(raw):
    var = {}
    raw2 = raw.copy()
    k=0
    for i,line in enumerate(raw):
        icomma = line.find(',')
        icolon = line.find(':')
        iequal = line.find('=')
        if icomma==-1 and icolon==-1 and iequal!=-1:
            var[line[:iequal]]=_str2val(line[iequal+1:],var)
            del raw2[i-k]
            k=k+1
    return var,raw2

In [7]:
test = rawlines[1:-6]
test.insert(2,'abc=0.2342345345')
test.insert(5,'ab=abc')
var,newtest=_rawlines2var(test)
print(set(test)-set(newtest))
test=newtest

{'ab=abc', 'abc=0.2342345345'}


In [8]:
var.items()

dict_items([('abc', 0.2342345345), ('ab', 0.2342345345)])

In [9]:
getElem = mli.getElem

import re
def _rawlines2cmd(raw,variables):
    cmds=[]
    raw2 = raw.copy()
    k=0
    for i,line in enumerate(raw):
        icolon = line.find(':')
        if icolon!=-1:
          del raw2[i-k]
          k=k+1
          line = re.split(':|,|=',line)
          name = line[0]
          cmd  = line[1]
          if cmd == 'sbend':
            f = getElem.sbend(name=name)
          elif cmd == 'beam':
            f = getElem.beam(name=name)
          elif cmd == 'units':
            f = getElem.units(name=name)
          elif cmd == 'globaldefaults':
            f = getElem.globaldefaults(name=name)
          elif cmd == 'autotrack':
            f = getElem.autotrack(name=name)
          elif cmd == 'particledump':
            f = getElem.particledump(name=name)
          elif cmd == 'iden':
            f = getElem.iden(name=name)
          elif cmd == 'end':
            f = getElem.end(name=name)
          elif cmd == 'raytrace':
            f = getElem.raytrace(name=name)
          elif cmd == 'ptm':
            f = getElem.ptm(name=name)
          elif cmd == 'tasm':
            f = getElem.tasm(name=name)
          elif cmd == 'stm':
            f = getElem.stm(name=name)
          elif cmd == 'gtm':
            f = getElem.gtm(name=name)
          elif cmd == 'tmo':
            f = getElem.tmo(name=name)
          elif cmd == 'tmi':
            f = getElem.tmi(name=name)
          elif cmd == 'nlinsert':
            f = getElem.nlinsert(name=name)
          elif cmd == 'monitor':
            f = getElem.monitor(name=name)
          elif cmd == 'marker':
            f = getElem.marker(name=name)
          elif cmd == 'drift':
            f = getElem.drift(name=name)
          elif cmd == 'quadrupole':
            f = getElem.quadrupole(name=name)
          elif cmd == 'vkicker':
            f = getElem.vkicker(name=name)
          elif cmd == 'sextupole':
            f = getElem.sextupole(name=name)
          elif cmd == 'sbend':
            f = getElem.sbend(name=name)
          elif cmd == 'dipedge':
            f = getElem.dipedge(name=name)
          elif cmd == 'srfc':
            f = getElem.srfc(name=name)
          if len(line)>2:
            for i in range(2,len(line),2):
              f[line[i]]=_str2val(line[i+1],variables)
            
          cmds.append(f)
    return cmds,raw2

In [10]:
cmd,newtest=_rawlines2cmd(test,var)

In [11]:
set(test) - set(newtest)

{'autotrack:autotrack,type=symplectic5',
 'beam:beam,particle=electron,energy=511000.1',
 'clear:iden',
 'fin:end',
 'gtm:gtm,nmap=1,iopt=2',
 'nlinsert:nlinsert,zstart=0.0,zend=1.8,steps=1000,zlen=1.8,k=1.45446332708327,c=0.01,tau=-0.4',
 'particledump:particledump,file=rays.out,precision=15,close=false,flush=true,nunits=0',
 'raytrace:raytrace,type=readonly,file1=rays.in',
 'setdefaults:globaldefaults,lfrngsbend=0,tfrngsbend=0,lfrngquad=0,tfrngquad=0,driftexact=0',
 'stm:stm,nmap=1',
 'tmi:tmi,iopt=2,ifile=16,nopt=1,nskip=0',
 'tmo:tmo,ifile=17',
 'units:units,type=static'}

In [12]:
print(set(test) and set(newtest))
test=newtest

{'linedump, line=(nlinsert gtm particledump ) '}


In [44]:
def _rawlines2line(raw,cmds):
    raw2=[]
    for line in raw:
        line=line.replace('&',' ')
        line=line.replace('(',' ')
        line=line.replace(')',' ')
        line=line.replace('=',' ')
        line=line.replace(',',' ')
        raw2 = raw2 + line.split()
    iElems = [i for i in range(len(raw2))]
    ilines = []
    elemList = raw2.copy()
    k=0
    for i,item in enumerate(raw2):
        if item == 'line':
            iElems.remove(i)
            iElems.remove(i-1)
            ilines.append(i)
            
    for i in iElems:
        for cmd in cmds:
            if cmd.name == raw2[i]:
                raw2[i]=cmd
             
    nlines = len(ilines)
    lines = [0]*nlines
    i=-1
    for i in range(nlines-1):
        lines[i]=getElem.line(name=raw2[ilines[i]-1],elemList=raw2[ilines[i]+1:ilines[i+1]])
    lines[i+1]=getElem.line(name=raw2[ilines[i+1]-1],elemList=raw2[ilines[i+1]+1:])
    return lines

In [43]:
line=_rawlines2line(test,cmd)

[1]
1
['linedump', 'line', 
    cmd: 'nlinsert'
   name: 'nlinsert'
 zstart: 0.0
   zend: 1.8
  steps: 1000
   zlen: 1.8
      k: 1.45446332708327
      c: 0.01
    tau: -0.4, 
  cmd: 'gtm'
 name: 'gtm'
 nmap: 1
 iopt: 2, 
       cmd: 'particledump'
      name: 'particledump'
      file: 'rays.out'
 precision: 15
     close: 'false'
     flush: 'true'
    nunits: 0]
[
    cmd: 'nlinsert'
   name: 'nlinsert'
 zstart: 0.0
   zend: 1.8
  steps: 1000
   zlen: 1.8
      k: 1.45446332708327
      c: 0.01
    tau: -0.4, 
  cmd: 'gtm'
 name: 'gtm'
 nmap: 1
 iopt: 2, 
       cmd: 'particledump'
      name: 'particledump'
      file: 'rays.out'
 precision: 15
     close: 'false'
     flush: 'true'
    nunits: 0]
